# In this notebook, we'll convert the raw timeseries to tSNR

In [73]:
import os
import numpy as np
import glob
import nibabel as nb
from scipy.stats import stats
from tqdm import tqdm


In [24]:
# define where wb_command is installed:
wb_home = '/home/vanes/Downloads/workbench/bin_rh_linux64/'
ts_home_dir = '/home/shared/2018/visual/HCP7TFIXED/'

data_dir = '/home/shared/2018/visual/hcp_cerebellum/'
ts_write_dir = os.path.join(data_dir,'timeseries')
resource_dir = os.path.join('/home','vanes','git','hcp_cerebellum','resources')


In [25]:
ts_files = glob.glob(os.path.join(ts_home_dir,'*'))
ss=os.listdir(ts_home_dir)
sd=np.array([os.path.isdir(os.path.join(ts_home_dir,s)) for s in ss])
dirs = np.array(ss)[sd]

In [8]:
# Loop over subjects and save subcortical niftis
# generic wb command:
wbc = wb_home + 'wb_command -cifti-separate {cii} COLUMN -volume-all {cii_n}.nii.gz'

for si, s in enumerate(np.sort(dirs)):

    fns = glob.glob(os.path.join(ts_home_dir,s,'*_clean.dtseries.nii'))
    for fn in fns:
        task = fn.split('/')[-1].split('_')[1]
        # convert to subcortical nifti
        save_fn =os.path.join(ts_write_dir,'sub_%d_%s'%(si,task))
        wbc_c = wbc.format(cii=fn, cii_n=save_fn)
        os.system(wbc_c)
        

In [30]:
dims = {
    'ang':0,
    'ecc':1,
    'gain':2,
    'meanvol':3,
    'r2':4,
    'rfsize':5
}

def create_roi_mask():
    """
    This mask deselects all voxels outside the retinotopic
    clusters identified in the average subject.
    """
    
    roi_fn = os.path.join(resource_dir,'volume_masks','cerebellum_retmaps.nii')
    maskimg = nb.load(roi_fn)
    roimask = (maskimg.get_data()>0) # valid voxels  

    return roimask

def determine_best_subjects():
    
    """
    This function determines the best subjects based on the median
    r-squared within the retinotopic clusters defined in the average subject.
    """

    roimask = create_roi_mask()

    all_r2 = []
    for sj in range(181):      

        # load the prf results nifti
        fn = os.path.join(data_dir,'all_subjects','prfresults_subject_%d.dscalar_data_sub.nii.gz'%sj)
        img = nb.load(fn)
        data = img.get_data()

        # mask r2s with roi mask:
        r2s = np.ravel(data[roimask,dims['r2']])    
        # and get median within these voxels
        all_r2.append(np.nanmedian(r2s))

    # sort subjects based on median r2
    best_subjects = np.argsort(all_r2)[::-1]

    return best_subjects

In [37]:
s = determine_best_subjects()


In [42]:
tasks = ['RETCON','RETEXP','RETBAR1','RETBAR2','RETCW','RETCCW']
# loop over subjects and compute tSNR
for si, sj in zip(np.arange(181),np.array(s)[these_s]):
    tsnrs = []
    for task in tasks:
        fn = os.path.join(ts_write_dir,'sub_%d'%sj+'_'+task+'.nii.gz')
        img = nb.load(fn)
        data = img.get_data()
        tsnr = np.mean(data,axis=-1) / np.std(data,axis=-1)
        tsnrs.append(tsnr)
    mean_tsnr = np.mean(tsnrs,axis=0)
    new_img = nb.Nifti1Image(mean_tsnr,affine=img.affine,header=img.header)
    save_fn = os.path.join(ts_write_dir,'sub_rank_%s_tsnr_avg_over_tasks.nii'%(si))
    nb.save(new_img,save_fn)
        

/home/vanes/anaconda2/envs/analysis/lib/python2.7/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in divide


# create tsnr map for different participant groups

In [63]:
sg={
# 186: range(0,5),
187: range(88,93),
# 184: range(176,181)
}

for s in sg.keys():
    tsnrs = []
    for si in sg[s]:
        fn = os.path.join(ts_write_dir,'sub_rank_%s_tsnr_avg_over_tasks.nii'%(si))
        img = nb.load(fn)
        tsnrs.append(img.get_data())
    mean_tsnr = np.mean(tsnrs,axis=0)
    print np.isnan(mean_tsnr).sum()
    new_img = nb.Nifti1Image(mean_tsnr,affine=img.affine,header=img.header)
    save_fn = os.path.join(ts_write_dir,'sub_rank_%d_tsnr_avg_over_tasks.nii'%(s))
    nb.save(new_img,save_fn)

    
    

870759


# correlate tsnr with participant rank for each voxel

In [77]:
tsnrs = []
ranks = []
for si in range(181):
    fn = os.path.join(ts_write_dir,'sub_rank_%s_tsnr_avg_over_tasks.nii'%(si))
    img = nb.load(fn)
    tsnr = img.get_data()
    tsnrs.append(tsnr)
    ranks.append(np.ones_like(tsnr)*si)

tsnrs = np.array(tsnrs)
ranks = np.array(ranks)

lenx = np.shape(tsnrs)[1]
leny = np.shape(tsnrs)[2]
lenz = np.shape(tsnrs)[3]

corrs = np.zeros((lenx,leny,lenz))
ps = np.zeros((lenx,leny,lenz))
for x in tqdm(range(lenx)):
    for y in range(leny):
        for z in range(lenz):
            r,p = stats.spearmanr(tsnrs[:,x,y,z],ranks[:,x,y,z])
#             if p < .05:
            corrs[x,y,z] = r
#             else:
#                 corrs[x,y,z] = np.nan
            ps[x,y,z] = p

100%|██████████| 91/91 [07:57<00:00,  5.18s/it]


In [78]:
new_img = nb.Nifti1Image(corrs,affine=img.affine,header=img.header)
save_fn = os.path.join(ts_write_dir,'tsnr_rank_corrs.nii')
nb.save(new_img,save_fn)

new_img = nb.Nifti1Image(ps,affine=img.affine,header=img.header)
save_fn = os.path.join(ts_write_dir,'tsnr_rank_corrs_p.nii')
nb.save(new_img,save_fn)

